In [15]:
import gym
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import random

In [13]:
"Tensorflow Version: ", tf.__version__, "Keras Version: ", tf.keras.__version__

('Tensorflow Version: ', '2.18.0', 'Keras Version: ', '3.8.0')

In [14]:
env = gym.make('CartPole-v1')
"Action Space: ", env.action_space, "Observation Space: ", env.observation_space

('Action Space: ',
 Discrete(2),
 'Observation Space: ',
 Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32))

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(24, input_shape=(4,), activation='relu'),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(2, activation='linear')
])

model.compile(optimizer='adam', loss='mse')

In [5]:
import gym
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

env = gym.make('Acrobot-v1', render_mode='human')

model = Sequential([
    Dense(128, input_shape=(env.observation_space.shape[0],), activation='relu'),
    Dense(64, activation='relu'),
    Dense(env.action_space.n, activation='linear')
])
model.compile(loss='mse', optimizer=Adam(), metrics=['mae'])

def train_model(n_episodes=500, epsilon=0.5, decay_factor=0.999, gamma=0.95, visualize=False):
    G_array = []
    for episode in range(n_episodes):
        observation = env.reset()[0]
        observation = np.array(observation).reshape(1, -1)

        epsilon *= decay_factor
        G = 0
        done = False

        while not done:
            if visualize:
                env.render()

            if np.random.random() < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(model.predict(observation, verbose=0).reshape(-1))

            new_observation, reward, done, _ = env.step(action)
            new_observation = np.array(new_observation).reshape(1, -1)

            target = reward + gamma * np.max(model.predict(new_observation, verbose=0).reshape(-1))
            target_vector = model.predict(observation, verbose=0).reshape(-1)
            target_vector[action] = target

            model.fit(observation, target_vector.reshape(1, -1), epochs=1, verbose=0)
            observation = new_observation
            G += reward

        G_array.append(G)
        print(f"Episode: {episode + 1}, Total Reward: {G}, Epsilon: {epsilon:.4f}")

    return G_array

train_model(visualize=True)
env.close()


ValueError: not enough values to unpack (expected 5, got 4)